# Question 2

# Finetuning GPT-2 

# Data Pre-processing

In [1]:
import json
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

import re
import html
import unicodedata

def clean_text(text):
    # 1. Normalize Unicode characters (Replace common Unicode characters)
    text = text.replace("\u201c", '"').replace("\u201d", '"').replace("\u2018", "'").replace("\u2019", "'")
    text = text.replace("\u2013", "-").replace("\u2026", "...").replace("\u2022", "*")  # Handle en dash, ellipsis, bullet points
    
    # 2. Decode HTML entities (e.g., "&amp;" becomes "&")
    text = html.unescape(text)
    
    # 3. Remove non-ASCII characters (if necessary, using a more aggressive approach)
    text = ''.join([c for c in text if ord(c) < 128])  # Keep only ASCII characters (lower-level cleaning)
    
    # 4. Remove unwanted symbols: Strip any non-letter, non-digit, non-punctuation characters
    text = re.sub(r'[^\w\s.,!?\'";:()-]', '', text)  # Keep common punctuation but remove others
    
    # 5. Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # 6. Remove leading and trailing spaces
    text = text.strip()
    
    # 7. Ensure proper sentence ending (optional, can be customized)
    if text and text[-1] not in ['.', '!', '?']:
        text += '.'  # Add period if no punctuation at the end

    return text


/home/vanshdhar/Desktop/AIDS/nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to structure the jokes
def structure_jokes_from_json(file_path):
    """
    Structure jokes from the original JSON file containing 'title' and 'body'.
    Assumes title is the question/prompt and body is the punchline/response.
    
    Args:
    - file_path (str): Path to the original JSON file containing the jokes.
    
    Returns:
    - list of structured jokes in the form of [{"text": "title <|sep|> body <|endofjoke|>"}]
    """
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    structured_jokes = []
    for entry in data:
        title = clean_text(entry["title"])
        body = clean_text(entry["body"])
        
        # Skip jokes that are too short
        if len(title.split()) < 3 or len(body.split()) < 3:
            continue
        
        # Structure joke as 'title <|sep|> body <|endofjoke|>'
        joke = f"{title} <|sep|> {body} <|endofjoke|>"
        structured_jokes.append({"text": joke})
    
    return structured_jokes

# Tokenize
Used the GPT2 tokenizer

Max_length used = 128

In [12]:
from datasets import Dataset
import torch
from transformers import GPT2Tokenizer

# Function to tokenize the jokes
def tokenize_data(dataset, tokenizer):
    """
    Tokenize the joke data for GPT-2 model.
    
    Args:
    - dataset (list): List of jokes to be tokenized.
    - tokenizer (GPT2Tokenizer): The GPT-2 tokenizer.
    
    Returns:
    - tokenized dataset
    """
    tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to be the end-of-text token

    def tokenize_function(examples):
        # Tokenize and ensure tensors are returned
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

    # Convert the list of structured jokes into a Dataset object from Hugging Face
    dataset = Dataset.from_list(dataset)
    
    # Tokenize the entire dataset
    return dataset.map(tokenize_function, batched=True, remove_columns=["text"])  # Remove original text column


# Driver Code

In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import torch
def fine_tune_gpt2(file_path, model_output_dir, epochs, batch_size, max_length):
    # Load the dataset and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    structured_jokes = structure_jokes_from_json(file_path)
    dataset = tokenize_data(structured_jokes, tokenizer)  # Ensure this returns tokenized dataset
    
    # Load the model
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    
    # Set the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_dir="./logs",
        save_steps=1000,
        save_total_limit=2,
        prediction_loss_only=True,
    )
    
    from torch.utils.data import DataLoader
    import torch

    # Custom collate function to handle the dataset
    def collate_fn(batch):
        input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch])
        attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch])
        labels = input_ids.clone()  # Shift input_ids for causal language modeling

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=collate_fn,  # Use the collate function here
    )

    # Start training
    trainer.train()

    # Save the fine-tuned model and tokenizer
    model.save_pretrained(model_output_dir)
    tokenizer.save_pretrained(model_output_dir)

In [14]:
# Fine-tune GPT-2 using the original JSON file containing jokes
fine_tune_gpt2(
    file_path="../data/reddit_jokes.json",  # Path to your original JSON file
    model_output_dir="../gpt2-joke-model",  # Directory to save the fine-tuned model
    epochs=1,  # Number of training epochs
    batch_size=4,  # Batch size
    max_length=128  # Maximum token length
)

Map: 100%|██████████| 155493/155493 [00:25<00:00, 6206.44 examples/s]
                                         
  0%|          | 0/38874 [03:58<?, ?it/s]            

{'loss': 1.2326, 'grad_norm': 1.4769753217697144, 'learning_rate': 4.935689664042805e-05, 'epoch': 0.01}


                                         
  0%|          | 0/38874 [05:41<?, ?it/s]             

{'loss': 1.2066, 'grad_norm': 2.4955990314483643, 'learning_rate': 4.87137932808561e-05, 'epoch': 0.03}


                                         
  0%|          | 0/38874 [07:25<?, ?it/s]             

...

{'loss': 1.0224, 'grad_norm': 1.3034898042678833, 'learning_rate': 1.1241446725317694e-06, 'epoch': 0.98}


                                           
  0%|          | 0/38874 [2:03:13<?, ?it/s]            

{'loss': 1.0358, 'grad_norm': 1.5755958557128906, 'learning_rate': 4.81041312959819e-07, 'epoch': 0.99}


                                           
100%|██████████| 38874/38874 [2:02:04<00:00,  5.31it/s]


{'train_runtime': 7324.2511, 'train_samples_per_second': 21.23, 'train_steps_per_second': 5.308, 'train_loss': 1.086230080627074, 'epoch': 1.0}


In [32]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("../gpt2-joke-model")
tokenizer = GPT2Tokenizer.from_pretrained("../gpt2-joke-model")

# Use the model
text = "What is the difference between"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_length = 100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is the difference between a black man and a pizza? <|sep|> A pizza can feed a family of four. <|endofjoke|>


# The model generates offensive and racist jokes often, because it was trained on the reddit jokes only.